In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline


In [2]:

# For time stamps
from datetime import datetime



In [3]:
# For division
from __future__ import division

In [4]:
end = datetime.now()
start = datetime(end.year -1 , end.month ,end.day)

In [8]:
from nsepy import get_history


ModuleNotFoundError: No module named 'nsepy'

In [ ]:
sbi = get_history(symbol="SBIN", start = start , end = end )
infosys = get_history(symbol="INFY", start = start , end = end )
pnb = get_history(symbol="PNB", start = start , end = end )
itc = get_history(symbol="ITC", start = start , end = end )


In [ ]:
sbi.info()

In [ ]:
sbi.head()

In [ ]:
sbi.describe()

In [ ]:
sbi['Close'].plot(legend= True , figsize =(10,4),color='purple')

In [ ]:
sbi['Close'].plot(legend= True , figsize =(10,4),color='purple',kind='kde')

In [ ]:
sbi['Volume'].plot(color='green')

In [ ]:
# Let's go ahead and plot out several moving averages
ma_day = [10,20,50]

for ma in ma_day:
    column_name = "MA for %s days" %(str(ma))
    sbi[column_name]=pd.rolling_mean(sbi['Prev Close'],ma)  # pandas has a built-in rolling mean calculator

In [ ]:
sbi[['Prev Close','MA for 10 days','MA for 20 days','MA for 50 days']].plot(subplots=False,figsize=(10,4))

In [ ]:
sbi[['Prev Close','MA for 10 days','MA for 20 days','MA for 50 days']].plot(subplots=True,figsize=(10,4))

In [ ]:
# We'll use pct_change to find the percent change for each day
sbi['Daily Return'] = sbi['Prev Close'].pct_change()

# Then we'll plot the daily return percentage
sbi['Daily Return'].plot(figsize=(25,6),legend=True,linestyle='--',marker='o')

In [ ]:
#overall look at the average daily return using a histogr
sbi['Daily Return'].hist(color="purple")

In [ ]:
# Note the use of dropna() here, otherwise the NaN values can't be read by seaborn
# use seaborn to create both a histogram and kde plot on the same figure.
sns.distplot(sbi['Daily Return'].dropna(),bins=10,color='red')


In [ ]:
closing_df=DataFrame()

In [ ]:
closing_df['sbi']=sbi['Close']
closing_df['pnb']=pnb['Close']
closing_df['infosys']=infosys['Close']
closing_df['itc']=itc['Close']

In [ ]:
closing_df.head()

In [ ]:

#  del closing_df['sbi']    --->to delete a column

In [ ]:
bank_rets = closing_df.pct_change()

In [ ]:
bank_rets.plot(figsize=(25,6),legend=True,marker='o')

In [ ]:
# We'll use joinplot to compare the daily returns 
sns.jointplot('sbi','sbi',bank_rets,kind='scatter')

In [ ]:
sns.jointplot('sbi','pnb',bank_rets,kind='reg' )

In [ ]:
sns.jointplot('sbi','pnb',bank_rets,kind='resid')

In [ ]:
sns.jointplot('sbi','pnb',bank_rets,kind='kde')

In [ ]:
sns.jointplot('sbi','pnb',bank_rets,kind='hex')

In [ ]:
# We can simply call pairplot on our DataFrame for an automatic visual analysis of all the compasns.pairplot(tech_rets.dropna())
sns.pairplot(bank_rets.dropna())

In [ ]:
# Set up our figure by naming it returns_fig, call PairPLot on the DataFrame
returns_fig = sns.PairGrid(bank_rets.dropna())

# Using map_upper we can specify what the upper triangle will look like.
returns_fig.map_upper(plt.scatter,color='purple')

# We can also define the lower triangle in the figure, inclufing the plot type (kde) or the color map (BluePurple)
returns_fig.map_lower(sns.kdeplot,cmap='cool_d')

# Finally we'll define the diagonal as a series of histogram plots of the daily return
returns_fig.map_diag(plt.hist,bins=30)


In [ ]:
# Set up our figure by naming it returns_fig, call PairPLot on the DataFrame
returns_fig = sns.PairGrid(closing_df)

# Using map_upper we can specify what the upper triangle will look like.
returns_fig.map_upper(plt.scatter,color='purple')

# We can also define the lower triangle in the figure, inclufing the plot type (kde) or the color map (BluePurple)
returns_fig.map_lower(sns.kdeplot,cmap='cool_d')

# Finally we'll define the diagonal as a series of histogram plots of the closing price
returns_fig.map_diag(plt.hist,bins=30)

In [ ]:
from IPython.display import SVG
SVG(url='http://upload.wikimedia.org/wikipedia/commons/d/d4/Correlation_examples2.svg')

In [ ]:
# Let's start by defining a new DataFrame as a clenaed version of the oriignal tech_rets DataFrame
rets = bank_rets.dropna()

area = np.pi*20 #np.pi funcs gives the value of pie , ie = 3.14159

plt.scatter(rets.mean(), rets.std(),alpha = 0.9,s =5*area) #alpha changes the brightness of the dots in the scatter plot
                                                            # area  is the radius of the dots d with the same centre
# Set the x and y limits of the plot (optional, remove this if you don't see anything in your plot)
plt.ylim([0.01,0.05])
plt.xlim([-0.005,0.005])

#Set the plot axis titles
plt.xlabel('Expected returns')
plt.ylabel('Risk')

# Label the scatter plots, for more info on how this is done, chekc out the link below
# http://matplotlib.org/users/annotations_guide.html
for label, x, y in zip(rets.columns, rets.mean(), rets.std()):
    plt.annotate(
        label, 
        xy = (x, y), xytext = (50, 50),
        textcoords = 'offset points', ha = 'left', va = 'top',
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=-0.3',))


In [ ]:
# Set up our time horizon
days = 365

# Now our delta
dt = 1/days

# Now let's grab our mu (drift) from the expected return data we got for sbi
mu = rets.mean()['sbi']

# Now let's grab the volatility of the stock from the std() of the average return
sigma = rets.std()['sbi']

In [ ]:
def stock_monte_carlo(start_price,days,mu,sigma):
    ''' This function takes in starting stock price, days of simulation,mu,sigma, and returns simulated price array'''
    
    # Define a price array
    price = np.zeros(days)
    price[0] = start_price
    # Shock and Drift
    shock = np.zeros(days)
    drift = np.zeros(days)
    
    # Run price array for number of days
    for x in xrange(1,days):
        
        # Calculate Schock
        shock[x] = np.random.normal(loc=mu * dt, scale=sigma * np.sqrt(dt))
        # Calculate Drift
        drift[x] = mu * dt
        # Calculate Price
        price[x] = price[x-1] + (price[x-1] * (drift[x] + shock[x]))
        
    return price

In [ ]:
# Get start price from sbi.head()
start_price = 308

for run in xrange(100):
    plt.plot(stock_monte_carlo(start_price,days,mu,sigma))
plt.xlabel("Days")
plt.ylabel("Price")  
plt.title('Monte Carlo Analysis for SBI')

In [ ]:
# Set a large numebr of runs
runs = 10000

# Create an empty matrix to hold the end price data
simulations = np.zeros(runs)

# Set the print options of numpy to only display 0-5 points from an array to suppress output
np.set_printoptions(threshold=5)

for run in xrange(runs):    
    # Set the simulation data point as the last stock price for that run
    simulations[run] = stock_monte_carlo(start_price,days,mu,sigma)[days-1];

In [ ]:
# Now we'lll define q as the 1% empirical qunatile, this basically means that 99% of the values should fall between here
q = np.percentile(simulations, 1)
    
# Now let's plot the distribution of the end prices
plt.hist(simulations,bins=200)

# Using plt.figtext to fill in some additional information onto the plot

# Starting Price
plt.figtext(0.6, 0.8, s="Start price: $%.2f" %start_price)
# Mean ending price
plt.figtext(0.6, 0.7, "Mean final price: $%.2f" % simulations.mean())

# Variance of the price (within 99% confidence interval)
plt.figtext(0.6, 0.6, "VaR(0.99): $%.2f" % (start_price - q,))

# Display 1% quantile
plt.figtext(0.15, 0.6, "q(0.99): $%.2f" % q)

# Plot a line at the 1% quantile result
plt.axvline(x=q, linewidth=4, color='r')

# Title
plt.title(u"Final price distribution for Sbi Stock after %s days" % days, weight='bold');

In [ ]:
for a in xrange(1,10,-2):
    print a

In [ ]:
for a in range(1,10,2):
    print a